In [112]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import log_loss
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn import ensemble, metrics, model_selection as ms, preprocessing, tree
from sklearn import linear_model as lm
import statsmodels.api as sm
import statsmodels.formula.api as smf
pd.set_option('display.max_columns', 20)
from sklearn.pipeline import Pipeline
from os import listdir
from os.path import isfile, join
from scipy.stats import poisson

In [2]:
data_folder = "C:/Users/Byron/Documents/Football Predictions/UnderstatPredictions/Datasets"

In [5]:
#read in files
team_stats_df = pd.read_csv(data_folder+'/UnderstatAttDefStr.csv') #contains 2014 - 2018 team stats
results_df = pd.read_csv(data_folder+'/UnderstatResultsAll.csv') #contains 2014 - 2018 team stats

In [41]:
team_stats_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87648 entries, 0 to 87647
Data columns (total 29 columns):
league                87648 non-null object
season                87648 non-null object
matchid               87648 non-null int64
matchdate             87648 non-null object
fixture               87648 non-null object
Team                  87648 non-null object
Opponent              87648 non-null object
GoalsFor              82872 non-null float64
GoalsAgainst          82872 non-null float64
xG                    82872 non-null float64
xGA                   82872 non-null float64
season_id             87648 non-null int64
league_teams_count    87648 non-null int64
season_gw             87648 non-null int64
Points                87648 non-null int64
AwayPoints            87648 non-null int64
HomeGame              87648 non-null int64
season_team_id        87648 non-null int64
xG_av                 84724 non-null float64
xGA_av                84724 non-null float64
GoalsFor_av  

In [8]:
results_df.head()

,league,season,matchid,matchdate,fixture,hometeam,awayteam,homegoals,awaygoals,hxG,axG,season_id,league_teams_count,season_gw,HomePoints,AwayPoints
0,Bundesliga,2014/15,5447,2014-08-22 19:30:00,Bayern Munich vs Wolfsburg,Bayern Munich,Wolfsburg,2.0,1.0,2.57012,1.198420,1,18,1,3,0
1,Bundesliga,2014/15,5448,2014-08-23 14:30:00,Hoffenheim vs Augsburg,Hoffenheim,Augsburg,2.0,0.0,1.52873,0.280777,2,18,1,3,0
2,Bundesliga,2014/15,5449,2014-08-23 14:30:00,Hannover 96 vs Schalke 04,Hannover 96,Schalke 04,2.0,1.0,1.17979,0.956665,3,18,1,3,0
3,Bundesliga,2014/15,5450,2014-08-23 14:30:00,Hertha Berlin vs Werder Bremen,Hertha Berlin,Werder Bremen,2.0,2.0,1.75585,1.194530,4,18,1,1,1
4,Bundesliga,2014/15,5451,2014-08-23 14:30:00,Eintracht Frankfurt vs Freiburg,Eintracht Frankfurt,Freiburg,1.0,0.0,1.75331,1.380840,5,18,1,3,0


In [7]:
team_stats_df.groupby(['league','season','HomeGame'])['xG','xGA','GoalsFor','GoalsAgainst'].mean()

xG       xGA  GoalsFor  GoalsAgainst
league     season  HomeGame                                            
Bundesliga 2014/15 0         1.119094  1.505023  1.166667      1.588235
                   1         1.505023  1.119094  1.588235      1.166667
           2015/16 0         1.231671  1.592789  1.264706      1.565359
                   1         1.592789  1.231671  1.565359      1.264706
           2016/17 0         1.177101  1.576887  1.209150      1.656863
                   1         1.576887  1.177101  1.656863      1.209150
           2017/18 0         1.134068  1.538006  1.192810      1.601307
                   1         1.538006  1.134068  1.601307      1.192810
           2018/19 0         1.406968  1.778001  1.388889      1.790850
                   1         1.778001  1.406968  1.790850      1.388889
           2019/20 0         1.453254  1.716249  1.512195      1.751220
                   1         1.716249  1.453254  1.751220      1.512195
EPL        2014/15 0         1.064662  1.478771  1.092105      1.473684
                   1         1.478771  1.064662  1.473684      1.092105
           2015/16 0         1.160057  1.421535  1.207895      1.492105
                   1         1.421535  1.160057  1.492105      1.207895
           2016/17 0         1.129919  1.458365  1.202632      1.597368
                   1         1.458365  1.129919  1.597368      1.202632
           2017/18 0         1.113645  1.496798  1.147368      1.531579
                   1         1.496798  1.113645  1.531579      1.147368
           2018/19 0         1.262907  1.592984  1.252632      1.568421
                   1         1.592984  1.262907  1.568421      1.252632
           2019/20 0         1.322038  1.560295  1.252788      1.498141
                   1         1.560295  1.322038  1.498141      1.252788
La Liga    2014/15 0         1.101853  1.517071  1.118421      1.536842
                   1         1.517071  1.101853  1.536842      1.118421
           2015/16 0         1.180560  1.581195  1.126316      1.618421
                   1         1.581195  1.180560  1.618421      1.126316
           2016/17 0         1.056833  1.449108  1.278947      1.663158
                   1         1.449108  1.056833  1.663158      1.278947
           2017/18 0         1.126874  1.624237  1.147368      1.547368
                   1         1.624237  1.126874  1.547368      1.147368
           2018/19 0         1.187737  1.590512  1.134211      1.452632
                   1         1.590512  1.187737  1.452632      1.134211
           2019/20 0         1.105861  1.550178  1.032129      1.530120
                   1         1.550178  1.105861  1.530120      1.032129
Ligue 1    2014/15 0         1.019974  1.355645  1.081579      1.410526
                   1         1.355645  1.019974  1.410526      1.081579
           2015/16 0         1.020544  1.383122  1.089474      1.436842
                   1         1.383122  1.020544  1.436842      1.089474
           2016/17 0         1.092802  1.454963  1.068421      1.547368
                   1         1.454963  1.092802  1.547368      1.068421
           2017/18 0         1.101148  1.462624  1.189474      1.528947
                   1         1.462624  1.101148  1.528947      1.189474
           2018/19 0         1.104206  1.447218  1.086842      1.471053
                   1         1.447218  1.104206  1.471053      1.086842
           2019/20 0         1.045975  1.386704  1.042308      1.484615
                   1         1.386704  1.045975  1.484615      1.042308
Serie A    2014/15 0         1.129405  1.453657  1.194737      1.500000
                   1         1.453657  1.129405  1.500000      1.194737
           2015/16 0         1.086451  1.430215  1.105263      1.471053
                   1         1.430215  1.086451  1.471053      1.105263
           2016/17 0         1.202604  1.552246  1.289474      1.665789
                   1         1.552246  1.202604  1.665789      1.289474
   

In [87]:
def season_average(df):
    '''
    Creates rolling season averages for homegoals and awaygoals
    '''
    season_av_df = df.groupby(['league','season','season_gw']).agg({"homegoals":"sum"
                                                                       ,"awaygoals":"sum"
                                                                       ,"fixture":"nunique"}).reset_index()

    season_totals = season_av_df.groupby(['league','season'])["homegoals","awaygoals","fixture"].transform(lambda row: row.rolling(window=36,min_periods=2).sum()).shift(1)

    league_totals = season_av_df.groupby(['league'])["homegoals","awaygoals","fixture"].transform(lambda row: row.rolling(window=36,min_periods=2).sum()).shift(1)

    season_av_df[["TotalHomeGoals","TotalAwayGoals","TotalFixtures"]] = season_totals
    season_av_df["SeasonHG"]=season_av_df["TotalHomeGoals"]/test_df["TotalFixtures"]
    season_av_df["SeasonAG"]=season_av_df["TotalAwayGoals"]/season_av_df["TotalFixtures"]
    season_av_df = season_av_df[["league","season","season_gw","SeasonHG","SeasonAG"]]
    return season_av_df


In [45]:
#results_df.groupby(['league','season','season_gw'])['homegoals'].mean()

test_df = team_stats_df.copy()
#predictions_df['ewma'] = pd.DataFrame.ewm(predictions_df["prices"], span=60).mean()
test_df["test_gw_av"] = (team_stats_df.groupby(['league','season','season_gw',"HomeGame"])["GoalsFor"].
 transform(lambda row: row.rolling(window=36, min_periods=2).mean()))
test_df[test_df["season_gw"]==3]


,league,season,matchid,matchdate,fixture,Team,Opponent,GoalsFor,GoalsAgainst,xG,...,GoalsFor_av,GoalsAgainst_av,AverageMethod,xG_av_gw,xGA_av_gw,GoalsFor_av_gw,GoalsAgainst_av_gw,AttStr,DefStr,test_gw_av
2,Bundesliga,2014/15,5418,2014-09-14 14:30:00,Eintracht Frankfurt vs Augsburg,Augsburg,Eintracht Frankfurt,1.0,0.0,1.000280,...,1.027778,2.513889,EMA No reset,1.289009,1.289009,1.250,1.250,0.506239,1.494818,NaN
36,Bundesliga,2014/15,5411,2014-09-12 19:30:00,Bayer Leverkusen vs Werder Bremen,Bayer Leverkusen,Werder Bremen,3.0,3.0,1.983210,...,3.027778,1.027778,EMA No reset,1.289009,1.289009,1.250,1.250,1.442145,0.465965,NaN
70,Bundesliga,2014/15,5413,2014-09-13 14:30:00,Bayern Munich vs VfB Stuttgart,Bayern Munich,VfB Stuttgart,2.0,0.0,1.229870,...,1.486111,1.000000,EMA No reset,1.289009,1.289009,1.250,1.250,1.568556,0.973388,2.500000
104,Bundesliga,2014/15,5414,2014-09-13 14:30:00,Borussia Dortmund vs Freiburg,Borussia Dortmund,Freiburg,3.0,1.0,1.716420,...,1.541667,2.000000,EMA No reset,1.289009,1.289009,1.250,1.250,1.100198,0.833719,2.666667
138,Bundesliga,2014/15,5417,2014-09-13 17:30:00,Borussia M.Gladbach vs Schalke 04,Borussia M.Gladbach,Schalke 04,4.0,1.0,3.379550,...,0.486111,0.486111,EMA No reset,1.289009,1.289009,1.250,1.250,1.036885,0.682089,3.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87460,Serie A,2019/20,13114,2019-09-14 16:00:00,Napoli vs Sampdoria,Sampdoria,Napoli,0.0,2.0,1.697860,...,0.500000,3.500000,Moving Average Season reset,1.494479,1.494479,1.625,1.625,0.625762,1.407360,1.527778
87498,Serie A,2019/20,13116,2019-09-15 16:00:00,Roma vs Sassuolo,Sassuolo,Roma,2.0,4.0,0.849970,...,2.500000,1.500000,Moving Average Season reset,1.494479,1.494479,1.625,1.625,1.343234,1.042236,1.472222
87536,Serie A,2019/20,13118,2019-09-16 18:45:00,Torino vs Lecce,Torino,Lecce,1.0,2.0,1.551200,...,2.500000,1.500000,Moving Average Season reset,1.494479,1.494479,1.625,1.625,0.855477,1.065261,1.555556
87574,Serie A,2019/20,13113,2019-09-14 18:45:00,Inter vs Udinese,Udinese,Inter,0.0,1.0,0.250925,...,1.000000,1.500000,Moving Average Season reset,1.494479,1.494479,1.625,1.625,0.827986,0.618324,1.388889


In [47]:
test_df.groupby(["league","season","season_gw","HomeGame"])["test_gw_av"].agg(["min","max"])

min    max
league     season  season_gw HomeGame                 
Bundesliga 2014/15 1         0         0.500000  1.000
                             1         0.800000  1.400
                   2         0         1.333333  2.000
                             1         1.000000  3.000
                   3         0         0.500000  1.125
...                                         ...    ...
Serie A    2019/20 36        1              NaN    NaN
                   37        0              NaN    NaN
                             1              NaN    NaN
                   38        0              NaN    NaN
                             1              NaN    NaN

[2232 rows x 2 columns]

In [105]:
def home_vars_prep(teams_df,join_list,keep_list):
    new_df = teams_df[join_list + keep_list]
    new_df = new_df.rename(columns = {col: col + '_Home' for col in keep_list})
    return new_df

def away_vars_prep(teams_df,join_list,keep_list):
    new_df = teams_df[join_list + keep_list]
    new_df = new_df.rename(columns = {col: col + '_Away' for col in keep_list})
    return new_df

join_list = ['league','season','fixture','season_gw','AverageMethod']
keep_list = ["Team","xG_av","xGA_av","AttStr","DefStr"]

home_df = home_vars_prep(team_stats_df,join_list,keep_list)
away_df = away_vars_prep(team_stats_df,join_list,keep_list)

season_av_df = season_average(results_df)
#home_df.head()
#fixture_vars = pd.merge(home_df,away_df, on = join_list)
#.query("HomeGame==1").drop(columns="HomeGame")

single_line = results_df[["league","season","fixture","season_gw","hometeam","awayteam"]].copy()
single_line = single_line.rename(columns={"hometeam":"Team_Home"
                                         , "awayteam":"Team_Away"})

single_line = pd.merge(single_line,home_df, on=['league', 'season', 'fixture', 'season_gw', 'Team_Home'])
single_line = pd.merge(single_line,away_df, on=['league', 'season', 'fixture', 'season_gw', 'Team_Away',"AverageMethod"])
single_line = pd.merge(single_line,season_av_df, on=['league', 'season', 'season_gw'])


single_line.head()

    

,league,season,fixture,season_gw,Team_Home,Team_Away,AverageMethod,xG_av_Home,xGA_av_Home,AttStr_Home,DefStr_Home,xG_av_Away,xGA_av_Away,AttStr_Away,DefStr_Away,SeasonHG,SeasonAG
0,Bundesliga,2014/15,Bayern Munich vs Wolfsburg,1,Bayern Munich,Wolfsburg,EMA No reset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Bundesliga,2014/15,Bayern Munich vs Wolfsburg,1,Bayern Munich,Wolfsburg,Moving Average No reset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Bundesliga,2014/15,Bayern Munich vs Wolfsburg,1,Bayern Munich,Wolfsburg,EMA Season reset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Bundesliga,2014/15,Bayern Munich vs Wolfsburg,1,Bayern Munich,Wolfsburg,Moving Average Season reset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Bundesliga,2014/15,Hoffenheim vs Augsburg,1,Hoffenheim,Augsburg,EMA No reset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [111]:
def Predicted_HG(HAtt,ADef,LgHG):
    Pred_HG = HAtt*ADef*LgHG
    return Pred_HG

def Predicted_AG(HDef,AAtt,LgAG):
    Pred_AG = HDef*AAtt*LgAG
    return Pred_AG

single_line["Pred_HG"] = single_line.apply(lambda row: Predicted_HG(
                                            row["AttStr_Home"],row["DefStr_Away"],row["SeasonHG"]), axis =1)
single_line["Pred_AG"] = single_line.apply(lambda row: Predicted_AG(
                                            row["DefStr_Home"],row["AttStr_Away"],row["SeasonAG"]), axis =1)

single_line[single_line["season_gw"]==3].head(20)

,league,season,fixture,season_gw,Team_Home,Team_Away,AverageMethod,xG_av_Home,xGA_av_Home,AttStr_Home,DefStr_Home,xG_av_Away,xGA_av_Away,AttStr_Away,DefStr_Away,SeasonHG,SeasonAG,Pred_HG,Pred_AG
72,Bundesliga,2014/15,Bayer Leverkusen vs Werder Bremen,3,Bayer Leverkusen,Werder Bremen,EMA No reset,1.858938,0.600633,1.442145,0.465965,1.872740,1.521327,1.452852,1.180229,1.222222,1.277778,2.080298,0.865027
73,Bundesliga,2014/15,Bayer Leverkusen vs Werder Bremen,3,Bayer Leverkusen,Werder Bremen,Moving Average No reset,1.839755,0.597436,1.427263,0.463484,1.854410,1.527665,1.438632,1.185147,1.222222,1.277778,2.067408,0.852001
74,Bundesliga,2014/15,Bayer Leverkusen vs Werder Bremen,3,Bayer Leverkusen,Werder Bremen,EMA Season reset,1.858938,0.600633,1.442145,0.465965,1.872740,1.521327,1.452852,1.180229,1.222222,1.277778,2.080298,0.865027
75,Bundesliga,2014/15,Bayer Leverkusen vs Werder Bremen,3,Bayer Leverkusen,Werder Bremen,Moving Average Season reset,1.839755,0.597436,1.427263,0.463484,1.854410,1.527665,1.438632,1.185147,1.222222,1.277778,2.067408,0.852001
76,Bundesliga,2014/15,Hoffenheim vs Wolfsburg,3,Hoffenheim,Wolfsburg,EMA No reset,1.410921,1.428555,1.094578,1.108258,1.098941,2.069700,0.852547,1.605652,1.222222,1.277778,2.148069,1.207298
77,Bundesliga,2014/15,Hoffenheim vs Wolfsburg,3,Hoffenheim,Wolfsburg,Moving Average No reset,1.414105,1.397534,1.097048,1.084192,1.101630,2.083225,0.854633,1.616144,1.222222,1.277778,2.166985,1.183971
78,Bundesliga,2014/15,Hoffenheim vs Wolfsburg,3,Hoffenheim,Wolfsburg,EMA Season reset,1.410921,1.428555,1.094578,1.108258,1.098941,2.069700,0.852547,1.605652,1.222222,1.277778,2.148069,1.207298
79,Bundesliga,2014/15,Hoffenheim vs Wolfsburg,3,Hoffenheim,Wolfsburg,Moving Average Season reset,1.414105,1.397534,1.097048,1.084192,1.101630,2.083225,0.854633,1.616144,1.222222,1.277778,2.166985,1.183971
80,Bundesliga,2014/15,Bayern Munich vs VfB Stuttgart,3,Bayern Munich,VfB Stuttgart,EMA No reset,2.021883,1.254706,1.568556,0.973388,0.582065,1.838413,0.451560,1.426222,1.222222,1.277778,2.734243,0.561639
81,Bundesliga,2014/15,Bayern Munich vs VfB Stuttgart,3,Bayern Munich,VfB Stuttgart,Moving Average No reset,2.036700,1.253185,1.580051,0.972208,0.578042,1.853845,0.448439,1.438194,1.222222,1.277778,2.777401,0.557080


In [127]:
tdf = single_line[(single_line['league']=="Bundesliga") &
                 (single_line['season']=="2014/15")].copy()
tdf["Prob_HG"] = tdf["Pred_HG"].apply(lambda x: poisson(x).pmf(range(0,10)))
tdf["Prob_AG"] = tdf["Pred_AG"].apply(lambda x: poisson(x).pmf(range(0,10)))
tdf[tdf['season_gw']==3].head()


,league,season,fixture,season_gw,Team_Home,Team_Away,AverageMethod,xG_av_Home,xGA_av_Home,AttStr_Home,...,xG_av_Away,xGA_av_Away,AttStr_Away,DefStr_Away,SeasonHG,SeasonAG,Pred_HG,Pred_AG,Prob_HG,Prob_AG
72,Bundesliga,2014/15,Bayer Leverkusen vs Werder Bremen,3,Bayer Leverkusen,Werder Bremen,EMA No reset,1.858938,0.600633,1.442145,...,1.872740,1.521327,1.452852,1.180229,1.222222,1.277778,2.080298,0.865027,"[0.12489301601426786, 0.25981466395612607, 0.2...","[0.4210399746396352, 0.3642111558917844, 0.157..."
73,Bundesliga,2014/15,Bayer Leverkusen vs Werder Bremen,3,Bayer Leverkusen,Werder Bremen,Moving Average No reset,1.839755,0.597436,1.427263,...,1.854410,1.527665,1.438632,1.185147,1.222222,1.277778,2.067408,0.852001,"[0.12651329158827337, 0.26155457905266527, 0.2...","[0.4265606233748236, 0.36342998408548755, 0.15..."
74,Bundesliga,2014/15,Bayer Leverkusen vs Werder Bremen,3,Bayer Leverkusen,Werder Bremen,EMA Season reset,1.858938,0.600633,1.442145,...,1.872740,1.521327,1.452852,1.180229,1.222222,1.277778,2.080298,0.865027,"[0.12489301601426786, 0.25981466395612607, 0.2...","[0.4210399746396352, 0.3642111558917844, 0.157..."
75,Bundesliga,2014/15,Bayer Leverkusen vs Werder Bremen,3,Bayer Leverkusen,Werder Bremen,Moving Average Season reset,1.839755,0.597436,1.427263,...,1.854410,1.527665,1.438632,1.185147,1.222222,1.277778,2.067408,0.852001,"[0.12651329158827337, 0.26155457905266527, 0.2...","[0.4265606233748236, 0.36342998408548755, 0.15..."
76,Bundesliga,2014/15,Hoffenheim vs Wolfsburg,3,Hoffenheim,Wolfsburg,EMA No reset,1.410921,1.428555,1.094578,...,1.098941,2.069700,0.852547,1.605652,1.222222,1.277778,2.148069,1.207298,"[0.11670933949795417, 0.2506996806808749, 0.26...","[0.29900403656923347, 0.3609870367726491, 0.21..."


In [140]:
tdf.iloc[1219,]

league                                                  Bundesliga
season                                                     2014/15
fixture                         Borussia Dortmund vs Werder Bremen
season_gw                                                       34
Team_Home                                        Borussia Dortmund
Team_Away                                            Werder Bremen
AverageMethod                          Moving Average Season reset
xG_av_Home                                                 1.54423
xGA_av_Home                                                0.98277
AttStr_Home                                                1.17339
DefStr_Home                                               0.746761
xG_av_Away                                                  1.2639
xGA_av_Away                                                1.49925
AttStr_Away                                               0.960382
DefStr_Away                                                1.1

In [158]:
tdf["score"] = tdf.apply(lambda row: pd.DataFrame(np.outer(row["Prob_HG"],row["Prob_AG"])*100), axis =1)
#tdf["score"] = tdf.apply(lambda row: (np.outer(row["Prob_HG"],row["Prob_AG"])*100), axis =1)
mx = tdf["score"][1219].values.max()
tdf["score"][1219].values ==mx
#pd.DataFrame(tdf["score"][1219])
#tdf.head()

#numpy.outer(numpy.array([1, 2]), numpy.array([3, 4]))

array([[False, False, False, False, False, False, False, False, False,
        False],
       [False, False, False, False, False, False, False, False, False,
        False],
       [ True, False, False, False, False, False, False, False, False,
        False],
       [False, False, False, False, False, False, False, False, False,
        False],
       [False, False, False, False, False, False, False, False, False,
        False],
       [False, False, False, False, False, False, False, False, False,
        False],
       [False, False, False, False, False, False, False, False, False,
        False],
       [False, False, False, False, False, False, False, False, False,
        False],
       [False, False, False, False, False, False, False, False, False,
        False],
       [False, False, False, False, False, False, False, False, False,
        False]])